# Example of Using Agent System

In [1]:
%cd "/Users/ainura/studies/thesis/llm-for-rec"
repo_path = "/Users/ainura/studies/thesis/llm-for-rec"

/Users/ainura/studies/thesis/llm-for-rec


Reading api keys

In [2]:
from dotenv import load_dotenv

load_dotenv(repo_path + "/api_keys.env")

True

## Install requirements

In [ ]:
!pip install -q -r '{repo_path}/requirements/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Add config

In [3]:
import os

config_dict = {
    "csv_args": {"delimiter": "\t"},
    "source_column": "item_id:token",
    "search_kwargs": {"k": 20},
    "data_path": os.path.join(repo_path, "datasets"),
    "load_col": {
        "inter": ["user_id", "item_id", "rating", "timestamp"],
        "item": ["item_id", "movie_title"],
    },
    "text_col": ["movie_title", "release_year", "class"],
    "MAX_ITEM_LIST_LENGTH": 10,
    "eval_args": {"split": {"LS": "valid_and_test"}, "order": "TO", "mode": "full"},
    "repeatable": True,
    "loss_type": "CE",
    "train_batch_size": 100,
    "eval_batch_size": 8,
    "valid_metric": "NDCG@10",
    "metrics": ["Recall", "NDCG"],
    "topk": [1, 20, 50, 100, 200],
    "train_neg_sample_args": None,
    "threshold_rating": 0.5,
}

## Get dataset and config

In [4]:
from llm4rec.tasks import RecBoleRetrievalRecommender
from llm4rec.dataset import RecboleSeqDataset
from recbole.data.utils import data_preparation
from recbole.config import Config
from recbole.model.abstract_recommender import AbstractRecommender
import torch

model_cls = RecBoleRetrievalRecommender
dataset_name = 'ml-100k'

config = Config(model=model_cls, dataset=dataset_name,
            config_dict=config_dict)

dataset = RecboleSeqDataset(config)
_, _, test_data = data_preparation(config, dataset)

/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/recbole/data/dataset/dataset.py:501: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[field].fillna(value="", inplace=True)
/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/recbole/data/dataset/dataset.py:1217: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  split_point = np.cumsum(feat[field].agg(len))[:-1]
/Users/ainura/miniconda3/envs/thesi

## Let's take first user

Getting batch and taking 1 interaction in that batch

In [5]:
i=0
batch = next(iter(test_data))
user = batch[0][i]
history_length = user['item_length']
history_names = test_data.dataset.item_id2text(user['item_id_list'][:history_length])
history_item_ids = test_data.dataset.id2token("item_id", user['item_id_list'][:history_length])
prev_interactions = dict(zip(history_item_ids, history_names))

Initializing the retreival model

In [6]:
retrieval_model = RecBoleRetrievalRecommender(config=config,
                                             dataset=dataset,
                                             items_info_path=os.path.join(config['data_path'], f'{dataset_name}.item'),
                                             csv_loader_args=dict(csv_args=config['csv_args'],
                                                                  source_column=config['source_column']),
                                             search_kwargs=config['search_kwargs'],
                                             search_type='similarity',
                                             emb_model_kwargs={"device":"cuda:0" if torch.cuda.is_available() else "cpu"})

/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Initializing the models for Ranker tool

In [7]:
from langchain_groq import ChatGroq


ranker_model = ChatGroq(model_name="llama3-70b-8192", temperature=0) 

### Creation of tools from the tasks

In [8]:
from llm4rec.tasks import RankerRecommender
from llm4rec.tools import create_retrieval_tool, create_dataset_item_dict_info_tool, create_ranking_tool


retrieval_tool = create_retrieval_tool(retrieval=retrieval_model)

dataset_info_dict_tool = create_dataset_item_dict_info_tool(test_data.dataset)

ranker = RankerRecommender(llm = ranker_model)
ranker_tool = create_ranking_tool(ranker=ranker)

### Initializing models for agents

In [9]:
agent_executor_model = ChatGroq(model_name="llama3-70b-8192", temperature=0) 
planning_model = ChatGroq(model_name="llama3-70b-8192", temperature=0) 
reflection_model = ChatGroq(model_name="llama3-70b-8192", temperature=0) 

# Simple Agent

In [10]:
from llm4rec.agents import SimpleAgent


agent = SimpleAgent(tools=[retrieval_tool], llm_executor=agent_executor_model) 

In [11]:
agent.recommend(user_profile="", prev_interactions=prev_interactions, top_k=config['search_kwargs']['k'])



> Entering new AgentExecutor chain...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Invoking: `retrieval_recommender` with `{'prev_interactions': {'172': 'movie_title: Empire Strikes Back, The; release_year: 1980; class: Action Adventure Drama Romance Sci-Fi War', '762': 'movie_title: Beautiful Girls; release_year: 1996; class: Drama', '235': 'movie_title: Mars Attacks!; release_year: 1996; class: Action Comedy Sci-Fi War', '546': 'movie_title: Broken Arrow; release_year: 1996; class: Action Thriller', '750': 'movie_title: Amistad; release_year: 1997; class: Drama', '147': 'movie_title: Long Kiss Goodnight, The; release_year: 1996; class: Action Thriller', '781': 'movie_title: French Kiss; release_year: 1995; class: Comedy Romance', '484': 'movie_title: Maltese Falcon, The; release_year: 1941; class: Film-Noir Mystery', '959': 'movie_title: Dazed and Confused; release_year: 1993; class: Comedy', '39': 'movie_title: Strange Days; release_year: 1995; class: Action Crime Sci-Fi'}, 'user_profile': 'User Profile', 'top_k': 20}`


['1378', '775', '265', '1649', '1101', '64

/Users/ainura/miniconda3/envs/thesis/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Here are the recommended items for the user:

1. Item 1378
2. Item 775
3. Item 265
4. Item 1649
5. Item 1101
6. Item 643
7. Item 152
8. Item 1241
9. Item 344
10. Item 1320
11. Item 212
12. Item 345
13. Item 257
14. Item 852
15. Item 1611
16. Item 1644
17. Item 1115
18. Item 264
19. Item 1237
20. Item 1197

> Finished chain.


{'input': "Task: User . This User has previous interactions with these items: {'172': 'movie_title: Empire Strikes Back, The; release_year: 1980; class: Action Adventure Drama Romance Sci-Fi War', '762': 'movie_title: Beautiful Girls; release_year: 1996; class: Drama', '235': 'movie_title: Mars Attacks!; release_year: 1996; class: Action Comedy Sci-Fi War', '546': 'movie_title: Broken Arrow; release_year: 1996; class: Action Thriller', '750': 'movie_title: Amistad; release_year: 1997; class: Drama', '147': 'movie_title: Long Kiss Goodnight, The; release_year: 1996; class: Action Thriller', '781': 'movie_title: French Kiss; release_year: 1995; class: Comedy Romance', '484': 'movie_title: Maltese Falcon, The; release_year: 1941; class: Film-Noir Mystery', '959': 'movie_title: Dazed and Confused; release_year: 1993; class: Comedy', '39': 'movie_title: Strange Days; release_year: 1995; class: Action Crime Sci-Fi'}. Please give 20 candidate items recommendations for this user considering hi

# Plan-and-Execute Agent

In [10]:
from llm4rec.agents import PlanExecuteAgent


agent_plan_execute = PlanExecuteAgent(tools=[retrieval_tool, ranker_tool, dataset_info_dict_tool], 
                                      llm_executor=agent_executor_model,
                                      llm_for_planning=planning_model,
                                      llm_for_reflection=reflection_model) 

In [13]:
response = agent_plan_execute.recommend(user_profile="", prev_interactions=prev_interactions, top_k=config['search_kwargs']['k'])



> Entering new AgentExecutor chain...

Invoking: `retrieval_recommender` with `{'prev_interactions': {'172': 'movie_title: Empire Strikes Back, The; release_year: 1980; class: Action Adventure Drama Romance Sci-Fi War', '762': 'movie_title: Beautiful Girls; release_year: 1996; class: Drama', '235': 'movie_title: Mars Attacks!; release_year: 1996; class: Action Comedy Sci-Fi War', '546': 'movie_title: Broken Arrow; release_year: 1996; class: Action Thriller', '750': 'movie_title: Amistad; release_year: 1997; class: Drama', '147': 'movie_title: Long Kiss Goodnight, The; release_year: 1996; class: Action Thriller', '781': 'movie_title: French Kiss; release_year: 1995; class: Comedy Romance', '484': 'movie_title: Maltese Falcon, The; release_year: 1941; class: Film-Noir Mystery', '959': 'movie_title: Dazed and Confused; release_year: 1993; class: Comedy', '39': 'movie_title: Strange Days; release_year: 1995; class: Action Crime Sci-Fi'}, 'user_profile': 'User Profile', 'top_k': 20}`


['

In [18]:
response[0]["output"]

'The top 20 candidate items recommended for the user are:\n\n1. Rhyme & Reason (1997) - Documentary\n2. Something to Talk About (1995) - Comedy Drama Romance\n3. Hunt for Red October, The (1990) - Action Thriller\n4. Big One, The (1997) - Comedy Documentary\n5. Six Degrees of Separation (1993) - Drama Mystery\n6. The Innocent (1994) - Drama Romance\n7. Sleeper (1973) - Comedy Sci-Fi\n8. Van, The (1996) - Comedy Drama\n9. Apostle, The (1997) - Drama\n10. Homage (1995) - Drama\n11. Unbearable Lightness of Being, The (1988) - Drama\n12. Deconstructing Harry (1997) - Comedy Drama\n13. Men in Black (1997) - Action Adventure Comedy Sci-Fi\n14. Bloody Child, The (1996) - Drama Thriller\n15. Intimate Relations (1996) - Comedy\n16. Sudden Manhattan (1996) - Comedy\n17. Twelfth Night (1996) - Comedy Drama Romance\n18. Mimic (1997) - Sci-Fi Thriller\n19. Twisted (1996) - Comedy Drama\n20. Family Thing, A (1996) - Comedy Drama'

### First the agent generated the Plan and then followed this plan. The Plan is:

In [20]:
response[1]["plan"]

["Retrieve similar candidate items based on user's previous interactions using retrieval_recommender",
 'Map item IDs to their attributes using item_dataset_pair_info',
 "Rank candidate items based on user's preferences using ranker_recommender",
 'Return top-ranked candidate items as recommendations for the user']

_Note_: The output of Agent can be changed, we will not return additional variable of plan. It was done just for the example.

The output of the agent will be in the dictionary format, where the answer is in response["output"]